In [1]:
spark

In [2]:
import pandas as pd
import numpy as np
from PIL import Image
from io import BytesIO
import re
# import tensorflow as tf
# import cv2

I will use the functionality of Image Datasource of spark to collect and process images in bytes, and then start processing the images for the neural network

In [3]:
images_folders = ['hdfs://172.24.144.178:9000/CA1/Images/Ireland/','hdfs://172.24.144.178:9000/CA1/Images/Honduran/']

In [4]:
images_rdd = spark.sparkContext.binaryFiles(','.join(images_folders))
# images_rdd = spark.sparkContext.binaryFiles('hdfs://172.24.144.178:9000/CA1/Images/Ireland/Ireland_001.jpeg')

In [5]:
images_rdd

org.apache.spark.api.java.JavaPairRDD@2e3d4f8b

In [6]:
def extract_data(data):
    file_path, image_data = data
#     image = Image.open(BytesIO(image_data))
#     image_array = np.array(image)
    
    file_name = file_path.split('/')[-1]
    file_name_without_ext = file_name.split(".")[0]
    label,name = file_name_without_ext.split('_')[0], file_name_without_ext.split('_')[1]
    
    return name, label, image_data

In [7]:
imageDf = images_rdd.map(lambda x: extract_data(x)).toDF(["Name","Label","Data"])

In [8]:
pandasImagesDF = imageDf.toPandas()

In [9]:
def processImage(data, target_size=(225,225)):
    imgbytes = BytesIO(data)
    image = Image.open(imgbytes)
    resized_img = image.resize(target_size, Image.ANTIALIAS)
    with BytesIO() as output:
        resized_img.save(output, format="PNG")
        new_image = Image.open(output)

        array = np.asarray(image).astype(np.float32) / 255
        return array

In [10]:
pandasImagesDF["Data"] = pandasImagesDF["Data"].apply(lambda x: processImage(x))

In [11]:
pandasImagesDF

,Name,Label,Data
0,001,Ireland,"[[[0.8, 0.67058825, 0.5019608], [0.7764706, 0...."
1,002,Ireland,"[[[0.3764706, 0.4862745, 0.57254905], [0.37647..."
2,003,Ireland,"[[[0.5137255, 0.5176471, 0.5372549], [0.513725..."
3,004,Ireland,"[[[0.29803923, 0.101960786, 0.011764706], [0.3..."
4,005,Ireland,"[[[0.5058824, 0.5568628, 0.5921569], [0.745098..."
5,006,Ireland,"[[[0.29803923, 0.24313726, 0.19215687], [0.298..."
6,007,Ireland,"[[[0.7529412, 0.8, 0.7921569], [0.74509805, 0...."
7,008,Ireland,"[[[0.7137255, 0.5254902, 0.22745098], [0.71372..."
8,009,Ireland,"[[[0.9529412, 0.9529412, 0.9529412], [0.952941..."
9,010,Ireland,"[[[0.6862745, 0.5803922, 0.5058824], [0.686274..."


In [12]:
pandasImagesDF = pandasImagesDF.sort_values(by=["Name"])

In [13]:
pandasImagesDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40 entries, 0 to 39
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    40 non-null     object
 1   Label   40 non-null     object
 2   Data    40 non-null     object
dtypes: object(3)
memory usage: 1.2+ KB


In [ ]:
pandasImagesDF

In [1]:
from platform import python_version

print(python_version())

3.8.10


In [14]:
import matplotlib.pyplot as plt

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/jose/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_43345/322768712.py", line 1, in <module>
    import matplotlib.pyplot as plt
  File "/home/jose/.local/lib/python3.8/site-packages/matplotlib/__init__.py", line 227, in <module>
    _check_versions()
  File "/home/jose/.local/lib/python3.8/site-packages/matplotlib/__init__.py", line 223, in _check_versions
    raise ImportError(f"Matplotlib requires {modname}>={minver}; "
ImportError: Matplotlib requires numpy>=1.20; you have 1.19.5

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/jose/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2105, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/home/jose/.local/lib/python3.8/site-packages/IPython/core

In [ ]:
np.array(pandasImagesDF["Data"][9]).astype("float32").shape

In [ ]:
# Determine the dimensions of the image
width =  225
height =  225

# Convert the byte array to a numpy array of uint8 data type
image_data = np.array(pandasImagesDF["Data"][0], dtype=np.uint8)
# image_data.reshape(-1,1)
# Reshape the array to match the dimensions of the image
# image_data = image_data.reshape((height, width))

# Create an image from the byte array
image = Image.fromarray(image_data)

# Display the image
image.show()

Importing from Keras functionality necessary to implement CNN

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from tensorflow.keras import utils
from sklearn.metrics import accuracy_score

2024-03-19 20:47:51.832382: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-03-19 20:47:51.832434: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


For Now there is only 20 photos per Category, the training size will be 28 and test 12, This is for testing. After this I'll get more images to improve the CNN

In [13]:
X_train = pandasImagesDF.iloc[:-12,2].values
y_train = pandasImagesDF.iloc[:-12,1].values
X_test = pandasImagesDF.iloc[-12:,2].values
y_test = pandasImagesDF.iloc[-12:,1].values


In [14]:
from sklearn.preprocessing import OneHotEncoder

In [15]:
encoder = OneHotEncoder(sparse_output=False)
y_train = np.array(y_train).reshape(-1, 1)
y_train = encoder.fit_transform(y_train)

y_test = np.array(y_test).reshape(-1,1)
y_test = encoder.fit_transform(y_test)

In [16]:
X_train = X_train.reshape(-1,1)
X_test = X_test.reshape(-1,1)

In [17]:
X_train.shape

(28, 1)

In [52]:
p = np.reshape(X_train[0][0], (225,225,3))

In [53]:
p.shape

(225, 225, 3)

In [18]:
X_train_reshaped = []
X_test_reshaped = []

for x in X_train:
#     x = x.reshape(-1,1)
    X_train_reshaped.append(x)
#     print(x[0])
#     print(x[0].shape)

for x in X_test:
#     x = x.reshape(-1,1)
    X_test_reshaped.append(x)
    

In [19]:
# X_train = np.reshape(X_train, (X_train.shape[0], 225,225,1))
X_test = np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
# y_train = np.reshape(y_train, (y_train.shape[0], 225,225,1))
y_test = np.reshape(y_test, (y_test.shape[0], y_test.shape[1],1))

In [20]:
X_train.shape

(28, 1)

Because the pixels are from 0 to 255, We have to normalize the pixels

In [21]:
# X_train_reshaped/=255
# X_test_reshaped/=255

In [22]:
model = Sequential()
model.add(Conv2D(25,kernel_size=(3,3),strides=(1,1),padding="valid", activation="relu", input_shape=(1080,1080,1)))
model.add(MaxPool2D(pool_size=(1,1)))
model.add(Flatten())

model.add(Dense(100, activation="relu"))
model.add(Dense(10,activation="softmax"))

Unexpected exception formatting exception. Falling back to standard exception


2024-03-19 20:48:42.090845: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2024-03-19 20:48:42.395319: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-19 20:48:42.395544: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2070 with Max-Q Design computeCapability: 7.5
coreClock: 1.125GHz coreCount: 36 deviceMemorySize: 8.00GiB deviceMemoryBandwidth: 322.46GiB/s
2024-03-19 20:48:42.395719: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-03-19 20:48:42.395808: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could

In [ ]:
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer="adam")

In [ ]:
y_train.shape

In [ ]:
z =np.(X_train)

In [ ]:
x_train = np.asarray(X_train)
Y_train = np.asarray(y_train)
x_test = np.asarray(X_test)
Y_test = np.asarray(y_test)

In [ ]:
model.fit(x_train, Y_train, batch_size=14, epochs=10, validation_data=(x_test, Y_test))